In [2]:
# I downgrade to tensorflow version 1.4.0 and keras version 2.0.8.
# https://github.com/keras-team/keras/issues/9621
! pip3 install tensorflow==1.15.0
! pip3 install keras==2.0.8

     |████████████████████████████████| 276kB 2.8MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.8 which is incompatible.
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [3]:
from google.colab import drive
drive.mount('/gdrive')
import matplotlib.pyplot as plt

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [4]:
! git clone -b tf2 https://github.com/calico/basenji.git

Cloning into 'basenji'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 6771 (delta 66), reused 60 (delta 30), pack-reused 6655
Receiving objects: 100% (6771/6771), 153.13 MiB | 12.59 MiB/s, done.
Resolving deltas: 100% (4899/4899), done.


In [5]:
%cd basenji

/content/basenji


In [6]:
! git checkout 24538f27b229a2d8fafef63af64b37fdac37fb60

Note: checking out '24538f27b229a2d8fafef63af64b37fdac37fb60'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 24538f2 ==


In [7]:
! pip install pyBigWig

     |████████████████████████████████| 71kB 2.3MB/s 
  Created wheel for pyBigWig: filename=pyBigWig-0.3.17-cp36-cp36m-linux_x86_64.whl size=177302 sha256=9b03cf5d53b85bed683b6a4a52ada95d6af7f1e26237478315fbecce5786a340
  Stored in directory: /root/.cache/pip/wheels/fb/27/2d/ac3e2e2d17894877fd3c4595ebd6fbd25ad805bfeab333f19b
Successfully built pyBigWig


In [8]:
! python setup.py develop

running develop
running egg_info
creating basenji.egg-info
writing basenji.egg-info/PKG-INFO
writing dependency_links to basenji.egg-info/dependency_links.txt
writing requirements to basenji.egg-info/requires.txt
writing top-level names to basenji.egg-info/top_level.txt
writing manifest file 'basenji.egg-info/SOURCES.txt'
writing manifest file 'basenji.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.6/dist-packages/basenji.egg-link (link to .)
Adding basenji 0.1 to easy-install.pth file

Installed /content/basenji
Processing dependencies for basenji==0.1
Searching for tensorflow==1.14
Reading https://pypi.org/simple/tensorflow/
Best match: tensorflow 1.14.0
Processing tensorflow-1.14.0-cp36-cp36m-manylinux1_x86_64.whl
Installing tensorflow-1.14.0-cp36-cp36m-manylinux1_x86_64.whl to /usr/local/lib/python3.6/dist-packages
Adding tensorflow 1.14.0 to easy-install.pth file
Installing freeze_graph script to /usr/local/bin
Installing saved_model_cli script to /usr/loc

In [0]:
import basenji

In [10]:
import tensorflow as tf
import keras

from tensorflow import keras
from tensorflow.keras.models import load_model

Using TensorFlow backend.


In [11]:
!pip install xlrd
import io
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload

auth.authenticate_user()
drive_service = build('drive', 'v3')

In [0]:
params = '1zRgN0BW6x3OxOu-LK_qZoLnEGC2wW_s9'
model     = '1rcRe_xKEMbAAXsMO5wT-lEVPmA4Igc8Z'
cell_1_motifs = '1sFgxm0S6UAawANhZ03YvfV1TrNafO8ig'
cell_2_motifs = '1LIJpLTDmFHYseWysi7lrXzsWCEzpNhDE'
cell_3_motifs = '1MYEx_J_aLS9k4IX-bG6vsNCphFoZsP-k'
cell_4_motifs = '1oKrEtaQD3Q8VcgQk6WtsG3GKrMM2FLIw'

file_ids = [params, model, cell_1_motifs, cell_2_motifs, cell_3_motifs, cell_4_motifs]
file_names = ['params.json', 'model.h5', "cell_1_motifs.pkl",  "cell_2_motifs.pkl",  "cell_3_motifs.pkl",  "cell_4_motifs.pkl"]

# Download the cell motifs
for i in list(range(len(file_ids))):
  request = drive_service.files().get_media(fileId=file_ids[i])
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
    _, done = downloader.next_chunk()

  downloaded.seek(0)
  with open(file_names[i], 'wb') as f:
    f.write(downloaded.read())

In [13]:
import numpy as np
import tensorflow as tf
import basenji
from basenji import seqnn
import json

with open('params.json') as params_open:
    params = json.load(params_open)
    
    params_model = params['model']
    params_train = params['train']
    
    model = seqnn.SeqNN(params_model)
    
    model.model.load_weights('model.h5')

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequence (InputLayer)        [(None, 1000, 4)]         0         
_________________________________________________________________
stochastic_shift (Stochastic (None, 1000, 4)           0         
_________________________________________________________________
gelu (GELU)                  (None, 1000, 4)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1000, 128)         8704      
_________________________________________________________________
batch_normalization (BatchNo (None, 1000, 128)         512       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 200, 128)          0         
_________________________________________________________________
gelu_1 (GELU)                (None, 200, 128)          0   

In [14]:
import pickle

CHAR_2_INDEX = {'A':[1, 0, 0, 0], 'C':[0, 1, 0, 0], 'G':[0, 0, 1, 0], 'T':[0, 0, 0, 1]}
motif_files = ["cell_1_motifs.pkl",  "cell_2_motifs.pkl",  "cell_3_motifs.pkl",  "cell_4_motifs.pkl"]

cell_1_motifs = []
cell_2_motifs = []
cell_3_motifs = []
cell_4_motifs = []

# Get cell motifs and store them for later use
curr_cell = 1
max_seq_len = 0
for file_name in motif_files:
  with open(file_name, 'rb') as f:
      data = pickle.load(f)
      for seq in data:
        max_seq_len = max(max_seq_len, len(seq))
        new_seq = []
        for letter in seq:
          new_seq.append(np.array(CHAR_2_INDEX[letter]))
        new_seq = np.array(new_seq)
        if curr_cell == 1:
          cell_1_motifs.append(new_seq)
        elif curr_cell == 2:
          cell_2_motifs.append(new_seq)
        elif curr_cell == 3:
          cell_3_motifs.append(new_seq)
        elif curr_cell == 4:
          cell_4_motifs.append(new_seq)
  curr_cell += 1
print(max_seq_len)
CELL_1_MOTIFS = np.array(cell_1_motifs)
CELL_2_MOTIFS = np.array(cell_2_motifs)
CELL_3_MOTIFS = np.array(cell_3_motifs)
CELL_4_MOTIFS = np.array(cell_4_motifs)

18


In [0]:
import numpy as np
CHAR_2_INDEX = {'A':[1, 0, 0, 0], 'C':[0, 1, 0, 0], 'G':[0, 0, 1, 0], 'T':[0, 0, 0, 1]}
CHARS = ["A", "C", "G", "T"]

SEQUENCE_LENGTH = 1000
TRAIN_DATA_LEN = 300
TEST_DATA_LEN = 100

A_STR_ARR = np.array(['A' for _ in range(SEQUENCE_LENGTH)])
C_STR_ARR = np.array(['C' for _ in range(SEQUENCE_LENGTH)])
G_STR_ARR = np.array(['G' for _ in range(SEQUENCE_LENGTH)])
T_STR_ARR = np.array(['T' for _ in range(SEQUENCE_LENGTH)])


MIN_MOTIF_OCCURRENCES = 1
MAX_MOTIF_OCCURRENCES = 3

MOTIF_OCCURRENCES = 10

DEBUG = False

def get_char_from_index(one_hot_vec):
  """
  :param one_hot_vec: the one hot vector
  :return: a string mapped to the one hot vector
  """
  for i in range(len(one_hot_vec)):
      if one_hot_vec[i] == 1:
          if i == 0:
              return "A"
          elif i == 1:
              return "C"
          elif i == 2:
              return "G"
          elif i == 3:
              return "T"
  print(one_hot_vec)
  raise(Exception("Improperly formatted vec"))


def randomly_add_in_motifs(sequence, cell_type):
    """
    :param sequence: the sequence to edit
    :return: a modified sequence with the motif inserted some x amount of times, the motif inserted, the index of the motif
    """
    # Randomly select a motif for this sequence.
    motif_selection = []
    if cell_type == 1:
      motif_selection = CELL_1_MOTIFS
    elif cell_type == 2:
      motif_selection = CELL_2_MOTIFS
    elif cell_type == 3:
      motif_selection = CELL_3_MOTIFS
    elif cell_type == 4:
      motif_selection = CELL_4_MOTIFS
    choices = np.arange(len(motif_selection))
    rand_int = np.random.choice(choices)
    min_index = int(rand_int)
    max_index = int((MOTIF_OCCURRENCES + rand_int) / len(motif_selection))
    motifs_to_use = motif_selection[min_index: max_index]
    
    for motif in motifs_to_use:
      # Get motif index
      count = 0
      for mot in motif_selection:
          if mot == motif_to_use:
              motif_index = count
              break
          count += 1
      assert(motif_selection[motif_index] == motif_to_use)
      orig_sequence = sequence
      # Randomly select the amount of times to repeat the motif.
      times_to_use_motif = np.random.randint(MIN_MOTIF_OCCURRENCES, MAX_MOTIF_OCCURRENCES)
      
      start_indices_used = set()
      while len(start_indices_used) < times_to_use_motif:
          start_index_to_use = np.random.randint(0, len(sequence) - len(motif_to_use))
          
          # Check for conflicts (i.e. do not overwrite a motif with the same motif, shifted)
          possible_conflicts = [start_index_to_use + i for i in range(len(motif_to_use))]
          regen_start = False
          for conflict in possible_conflicts:
              if conflict in start_indices_used:
                  regen_start = True
          if regen_start:
              continue
              
          start_indices_used.add(start_index_to_use)
          
          for i in range(len(motif_to_use)):
              sequence[start_index_to_use + i] = motif_to_use[i]
            
    return sequence


def gen_data(train_data_len=TRAIN_DATA_LEN, test_data_len=TEST_DATA_LEN, add_motifs=False, for_cell_type=1):
    """
    :param train_data_len: the length of the training data to return
    :param test_data_len: the length of the test data to return
    :return: train_sequences_x, train_sequences_y, test_sequences_x, test_sequences_y
    """
    train_sequences_x = []
    train_sequences_y = []
    test_sequences_x = []
    test_sequences_y = []
    
    for i in range(train_data_len + test_data_len):
        
        # Get 4 random integers.
        char_probabilities = abs(np.random.randn(len(CHAR_2_INDEX.keys()), 1))
        
        # Normalize the random integers to effectively get random probabilities for each A, C, G, T.
        char_probabilities /= np.sum(char_probabilities)
        
        # Get randomly initialized sequence. 
        sequence = np.random.rand(SEQUENCE_LENGTH)
        
        # Get arrays of booleans representing the positions of the letters in the sequence based on the random probs.
        A_mask = (sequence < char_probabilities[0])
        A = np.ma.masked_array(A_STR_ARR, mask=np.invert(A_mask))
        C_mask = (sequence < char_probabilities[1] + char_probabilities[0]) * (sequence >= char_probabilities[0])
        C = np.ma.masked_array(C_STR_ARR, mask=np.invert(C_mask))
        G_mask = (sequence < char_probabilities[2] + char_probabilities[1] + char_probabilities[0]) * (sequence >= char_probabilities[1] + char_probabilities[0])
        G = np.ma.masked_array(G_STR_ARR, mask=np.invert(G_mask))
        T_mask = (sequence >= char_probabilities[2] + char_probabilities[1] + char_probabilities[0])
        T = np.ma.masked_array(T_STR_ARR, mask=np.invert(T_mask))
        
        if False:
            print(A_mask)
            print(C_mask)
            print(G_mask)
            print(T_mask)
            
            for i in range(SEQUENCE_LENGTH):
                a = 1 if A_mask[i] == True else 0
                c = 1 if C_mask[i] == True else 0
                g = 1 if G_mask[i] == True else 0
                t = 1 if T_mask[i] == True else 0
                
                if False:
                    print(A_mask[i])
                    print(C_mask[i])
                    print(G_mask[i])
                    print(T_mask[i])

                # noinspection PyBroadException
                try:
                    assert(a + c + g + t == 1)
                except Exception as e:
                    print("A = " + str(bool(a)))
                    print("C = " + str(bool(c)))
                    print("G = " + str(bool(g)))
                    print("T = " + str(bool(t)))
                    print(i)
                    print(sequence[i])
                    print(char_probabilities)
                    return
                
        A = np.array(A.filled(fill_value=""))
        C = np.array(C.filled(fill_value=""))
        G = np.array(G.filled(fill_value=""))
        T = np.array(T.filled(fill_value=""))
        
        
        sequence = np.core.defchararray.add(np.core.defchararray.add(A, C), np.core.defchararray.add(G, T))
        
        if add_motifs:
          sequence = randomly_add_in_motifs(sequence, for_cell_type)
        
        if i < train_data_len:
            train_sequences_x.append(sequence)
            train_sequences_y.append(np.array([[0, 1, 0, 0]]))
        else:
            test_sequences_x.append(sequence)
            test_sequences_y.append(np.array([[0, 1, 0, 0]]))
            
        
        sequence = np.core.defchararray.add(np.core.defchararray.add(A, C), np.core.defchararray.add(G, T))
    
    return (np.array(one_hot_encode_sequence(train_sequences_x)), 
            np.array(train_sequences_y), 
            np.array(one_hot_encode_sequence(test_sequences_x)), 
            np.array(test_sequences_y))



            
def one_hot_encode_sequence(sequences):
  """
  :param sequences: the sequences to encode in a one-hot vector
  :return: one-hot vector encoded sequences
  """
  return [np.array([CHAR_2_INDEX[char] for char in seq]) for seq in sequences]

In [0]:
def eval_model(model, test_x_arr, test_y_arr):
  """
  :param model: the Basenji model passed
  :param test_x_arr: the test array to predict, the x values
  :param test_y_arr: the ground-truth values mapped to the x-input
  :return: the accuracy of the test data passed to the Basenji model
  """
  return model.predict(test_x_arr,  test_y_arr)

In [27]:
train_sequences_x, train_sequences_y, test_sequences_x, test_sequences_y = gen_data(add_motifs=True, for_cell_type=1)
test_sequences_x, test_sequences_y = np.array(test_sequences_x, dtype=np.int), np.array(test_sequences_y, dtype=np.int)

print(test_sequences_x.shape)
print(test_sequences_y.shape)

print(model.model.summary())
    
result = model.model.predict(test_sequences_x)
print("sdfsdf")
print("result = " + str(result))



(100, 1000, 4)
(100, 1, 4)
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequence (InputLayer)        [(None, 1000, 4)]         0         
_________________________________________________________________
stochastic_shift (Stochastic (None, 1000, 4)           0         
_________________________________________________________________
gelu (GELU)                  (None, 1000, 4)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1000, 128)         8704      
_________________________________________________________________
batch_normalization (BatchNo (None, 1000, 128)         512       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 200, 128)          0         
_________________________________________________________________
gelu_1 (GELU)                (No

In [0]:
DEBUG = False
NEG_FACTOR = -1/3


def get_err_vec(seq_predictions, desired, undesired, max_objective=False):
    """
    Calculates the error vector for the given predictions. 
    Implicitly contains the objective function that we are maximizing.
    
    :param seq_predictions: Predictions from which we calculate the error
    :param desired: an array of booleans, with one Truth value, indicating the cell to maximize
    :param undesired: an array of booleans, with one False value, indicated the cell to maximize
    :param max_objective: whether to use the max objective
    :return: The error for the given predictions
    """
    pos_mask = seq_predictions * desired

    if max_objective:
      count = 0
      neg_stuff = None
      for bool_val in undesired:
        if bool_val is True:
          mask_temp = [count == i for i in range(len(undesired))]
          neg_stuff_temp = np.sum(seq_predictions * mask_temp, axis=-1)
          
          if neg_stuff is not None:
            neg_stuff = np.maximum(neg_stuff, neg_stuff_temp)
          else:
            neg_stuff = neg_stuff_temp
        count += 1

      pos_stuff = np.sum(pos_mask, axis=-1)
      curr_err = np.subtract(pos_stuff, neg_stuff)
    else:
      neg_mask = seq_predictions * undesired * NEG_FACTOR
      curr_err = np.sum(pos_mask + neg_mask, axis=-1)
    return curr_err.reshape((curr_err.shape[0], 1, 1))


def convert_one_hot_seq_to_seq(one_hot_seq):
  """
  :param one_hot_seq: the one-hot encoded sequence
  :return: the sequence string
  """
  seq = ""
  for elem in one_hot_seq:
      seq += get_char_from_index(elem)
  return seq


def disp_highest_act(seq_err, seq_predictions, sequences, desired, undesired, is_last=False):
    """
    Displays the minimum error (highest cell-type specific activation) for the passed sequences, 
    given the error mask
    
    :param seq_err: the calculated error
    :param seq_predictions: the calculated activations
    :param sequences: the current sequences
    :param desired: an array of booleans, with one Truth value, indicating the cell to maximize
    :param undesired: an array of booleans, with one False value, indicated the cell to maximize
    :param is_last: whether to return the best sequence, or simply print the activations
    :return: None if not is_last, the best sequence if is_last is True
    """
    min_err = np.max(seq_err, axis=0)
    min_err_index = np.argmax(seq_err.ravel())
    min_err_seq_acts = seq_predictions[min_err_index]
    
    print("Current Best Activations: " + str(min_err_seq_acts[0]))
    activation = np.sum(desired * min_err_seq_acts + undesired * min_err_seq_acts * NEG_FACTOR)
    
    print("Current Best Activation: " + str(activation))
    if is_last:
        seq = sequences[min_err_index]
        print(convert_one_hot_seq_to_seq(seq))
        return seq


def calculate_loss_mask(curr_predictions, cached_predictions, desired, undesired, metropolis_search=False, max_objective=False):
    """
    Caculates the loss mask which is True if sequence should be kept. False otherwise.
    
    :param curr_predictions: the current predictions
    :param cached_predictions: the cached predictions
    :param desired: an array of booleans, with one Truth value, indicating the cell to maximize
    :param undesired: an array of booleans, with one False value, indicated the cell to maximize
    :param metropolis_search: whether to use the metropolis search
    :param max_objective: whether to use the max objective [i.e. cell activation - max(of other cells)], 
    or the cell_desired - (1/3)(sum of other cell activations) objective
    :return: The loss mask
    """
    curr_err = get_err_vec(curr_predictions, desired, undesired, max_objective=max_objective)
    cache_err = get_err_vec(cached_predictions, desired, undesired, max_objective=max_objective)

    should_keep_new = (curr_err <= cache_err)
    if metropolis_search:
      delta_g = cache_err - curr_err
      delta_g *= -1
      metrop_prob = np.exp(delta_g)
      probs = np.random.uniform(low=0, high=1, size=metrop_prob.shape)
      accept = probs <= metrop_prob
      should_keep_new = np.logical_or(should_keep_new, accept)

    temp = (curr_err <= cache_err)
    
    return should_keep_new


def get_new_sequences(curr_predictions, cached_predictions, curr_sequences, cached_sequences, desired, undesired, metropolis_search=False, kill_off_pop=False, max_objective=False):
    """
    Merges the newly mutated curr_sequences with the cached sequences based on the current and prior cell activations
    
    :param curr_predictions: the Basenji-calculated activations for the current predictions
    :param cached_predictions: the predictions, as above, but from the previous sequences (before the most recent mutation)
    :param curr_sequences: the newly mutated sequences
    :param cached_sequences: the cached sequences from the previous iteration (after merging)
    :param desired: an array of booleans, with one Truth value, indicating the cell to maximize
    :param undesired: an array of booleans, with one False value, indicated the cell to maximize
    :param metropolis_search: whether to use the metropolis search
    :param kill_off_pop: whether to cull the population for False values in the loss mask or revert them back to the original sequences
    :param max_objective: whether to use the max objective [i.e. cell activation - max(of other cells)], 
    or the cell_desired - (1/3)(sum of other cell activations) objective
    :return: the merged sequences containing <= overall error per sequence than each sequence in cached_sequences
    """
    loss_mask = calculate_loss_mask(curr_predictions, cached_predictions, desired, undesired, metropolis_search=metropolis_search, max_objective=max_objective)
    if kill_off_pop:
      loss_mask = loss_mask.ravel()
      merged = np.compress(loss_mask, curr_sequences, axis=0)
    else:
      merged = np.where(loss_mask, curr_sequences, cached_sequences)
    if DEBUG:
        for i in range(len(curr_predictions)):
            loss = curr_predictions[i][0][1] + NEG_FACTOR * (curr_predictions[i][0][0] + curr_predictions[i][0][2] + curr_predictions[i][0][3])
            cached_loss = cached_predictions[i][0][1] + NEG_FACTOR * (cached_predictions[i][0][0] + cached_predictions[i][0][2] + cached_predictions[i][0][3])
            # noinspection PyBroadException
            if loss == cached_loss:
                assert check_mutation(curr_sequences[i], cached_sequences[i])
            try:
                assert loss != cached_loss
            except Exception as e:
                print(loss)
                print(cached_loss)
                print(merged[i])
                raise e
    return merged, loss_mask
    

def mutate(sequences, cached_muts_map):
    """
    Mutates the passed sequences by randomly replacing one base pair (with a one-hot vector)

    :param sequences: the sequences to mutate
    :param cached_muts_map: cached the mutations for debugging purposes, later checked with check_mutation(...)
    :return: the mutated sequences
    """
    new_seqs = np.zeros(sequences.shape)
    cached_muts_map = {}
    for i in range(len(sequences)):
        
        new_seqs[i] = sequences[i]
        
        char_set = set(CHARS)
        index_to_get = np.random.randint(SEQUENCE_LENGTH)
        char_to_remove = get_char_from_index(sequences[i][index_to_get])
        char_set.remove(char_to_remove)
        choice = CHAR_2_INDEX[np.random.choice(list(char_set))]
        assert get_char_from_index(np.array(choice)) != get_char_from_index(sequences[i][index_to_get])
        
        new_seqs[i][index_to_get] = np.array(choice)
        cached_muts_map[i] = index_to_get
        
        assert get_char_from_index(np.array(choice)) == get_char_from_index(new_seqs[i][index_to_get])
        
    return new_seqs


def check_mutation(new_seq, prev_seq):
  """
  :param new_seq: the new sequence being checked
  :param prev_seq: the sequence before the mutation
  :return: whether or not the sequence was properly mutated
  """
  is_diff = False
  for i in range(len(new_seq)):
      if is_diff is True:
          if get_char_from_index(new_seq[i]) != get_char_from_index(prev_seq[i]):
              print(get_char_from_index(new_seq[i]))
              print(get_char_from_index(prev_seq[i]))
              print(new_seq)
              print(prev_seq)
              raise(Exception("The sequence did not mutate. There is a bug."))
      is_diff = is_diff or (get_char_from_index(new_seq[i]) != get_char_from_index(prev_seq[i]))
  return is_diff


def check_mutations(new_seqs, prev_seqs, cached_muts_map):
  """
  :param new_seq: the new sequence being checked
  :param prev_seq: the sequence before the mutation
  :param cached_muts_map: the cached mutations from mutate(...)
  :return: nothing, but raises Exception if there is an error
  """
  for i in range(len(new_seqs)):
      result = check_mutation(new_seqs[i], prev_seqs[i])
      try:
          assert result is True
      except Exception as e:
          print(cached_muts_map[i])
          print(new_seqs[i][cached_muts_map[i]])
          print(prev_seqs[i][cached_muts_map[i]])
          raise e
            
            
def check_mask(mask, new_seqs, prev_seqs):
  """
  Debugging function 
  :param mask: the loss mask
  :param new_seq: the new sequence being checked
  :param prev_seq: the sequence before the mutation
  :return: -
  """
  for i in range(len(mask)):
      if mask[i] is True:
          result = check_mutation(new_seqs[i], prev_seqs[i])
          assert result is True
      else:
          pass
            

def get_cell_type_desired_outcome(cell_type):
  """
  Example: The desired outcome for cell type 1 (index 0) is [True, False, False, False]
  :param cell_type: the cell type for which to get the desired outcome boolean array
  :return: the desired outcome boolean array
  """
  return [bool(cell_type == 1), bool(cell_type == 2), bool(cell_type == 3), bool(cell_type == 4)]


def get_cell_type_undesired_outcome(cell_type):
  """
  Example: The undesired outcome for cell type 1 (index 0) is [False, True, True, True]
  :param cell_type: the cell type for which to get the undesired outcome boolean array
  :return: the undesired outcome boolean array
  """
  return [not bool(cell_type == 1), not bool(cell_type == 2), not bool(cell_type == 3), not bool(cell_type == 4)]


def simulate(for_iterations=3, sequences_to_return=10, metropolis_search=False, add_motifs=False, max_objective=False, kill_off_pop=False):
    """
    Example: The undesired outcome for cell type 1 (index 0) is [False, True, True, True]
    :param for_iterations: the cell type for which to get the undesired outcome boolean array
    :param sequences_to_return: the amount of sequences per cell to return, total sequences returned = cells * sequences_to_return
    :param metropolis_search: whether to use the metropolis search
    :param add_motifs: whether to initialize the sequences with motifs
    :param max_objective: whether to use the max objective [i.e. cell activation - max(of other cells)], 
    or the cell_desired - (1/3)(sum of other cell activations) objective
    :param kill_off_pop: whether to cull the population for False values in the loss mask or revert them back to the original sequences
    :return: the undesired outcome boolean array
    """

    # Initialize the simulation varaibles
    cached_muts_map = {}
    output_seqs = []
    cell_types = [i + 1 for i in range(4)]
    
    # Tracking time until completion
    from time import time
    start_time = int(time())
    
    # Iterate through the cell types
    for cell in cell_types:
        
        desired_outcome = get_cell_type_desired_outcome(cell)
        undesired_outcome = get_cell_type_undesired_outcome(cell)
        cell_seqs = []
        
        for seq_num in range(sequences_to_return):
            print("CELL_TYPE = " + str(cell))
            print("SEQUENCE NUMBER = " + str(seq_num))
            
            total_seqs = len(cell_types) * sequences_to_return
            curr_time = int(time())
            seqs_done = (seq_num + sequences_to_return * (cell - 1))
            if seqs_done > 0:
                progress = seqs_done / total_seqs
                print("Progress = " + str(progress) + "%")
                time_per_seq = (curr_time - start_time) / seqs_done
                print("Expected Time Left: " + str(time_per_seq * (total_seqs - seqs_done)) + "s")
            
            # Generate the data for this sequence iteration
            _, _, test_sequences_x, _ = gen_data(add_motifs=add_motifs, for_cell_type=cell)
            test_sequences_x = np.array(test_sequences_x, dtype=np.int)
            prev_sequences = np.array(list(test_sequences_x))
                  
            for i in range(for_iterations):
                
                test_sequences_x = mutate(prev_sequences, cached_muts_map)

                new_result = model.model.predict(test_sequences_x)
                prev_result = model.model.predict(prev_sequences)

                err_vec = get_err_vec(prev_result, desired_outcome, undesired_outcome, max_objective=max_objective)
                
                if i == for_iterations - 1:
                  # The sequence is complete given the iterations passed
                  seq = disp_highest_act(err_vec, prev_result, prev_sequences, desired_outcome, undesired_outcome, is_last=True)
                  if seq is not None:
                      print(seq)
                      cell_seqs.append(seq)

                if DEBUG:
                  check_mutations(test_sequences_x, prev_sequences, cached_muts_map)

                prev_sequences, loss_mask = get_new_sequences(new_result, prev_result, prev_sequences, test_sequences_x, desired_outcome, undesired_outcome, metropolis_search=metropolis_search, max_objective=max_objective, kill_off_pop=kill_off_pop)

                if DEBUG:
                    check_mask(loss_mask, test_sequences_x, prev_sequences)
        output_seqs.append(cell_seqs)
    return np.array(output_seqs)

In [73]:
files_to_use = ["metrop_no_motifs_cull", "metrop_with_motifs_cull", "metrop_motifs_max_objc_cull", "metrop_no_motifs_max_objc_cull", "metrop_nwith_motifs_max_objc_cull"]
configs = [
           [files_to_use[0], True, False, False, False], 
           [files_to_use[1], True, True, False, False], 
           [files_to_use[2], True, True, True, False],
           [files_to_use[3], True, False, True, False],
           [files_to_use[4], True, True, True, True]
           ]

assert(len(configs) == len(files_to_use))

for config in configs:
  print("STARTING FILE #" + str(config[0]))
  seqs = simulate(for_iterations=1000, sequences_to_return=20, metropolis_search=config[1], add_motifs=config[2], max_objective=config[3], kill_off_pop=config[4])
  print(seqs) 
  np.save(config[0], seqs)

STARTING FILE #metrop_nwith_motifs_max_objc_cull
CELL_TYPE = 1
SEQUENCE NUMBER = 0
Current Best Activations: [3.1822958 2.7665093 2.7135785 1.1761011]
Current Best Activation: 0.9635662
ATGGTATTTTAGTTGTTTTTGGGCAGTATATTAAAATTAATATTCGGGTGGTTCTTTTAGTTTTAATGAGACGATAGGAGATATTGTAATAAAATATTTTTTGAAATGATTCGTTTTAGTAGAGAATTATAGAAATGTTATAGATTAGGTGTTGTATGGATTGAGATGTATAAAGTTAGTATATATATTTATTCTATCTTTAATGTAATCAATGTTGATAGGGTGTTTGTCTCAGATATAATTTGGTTTTATAGTTATTTGTGATTTGAATGGCAAGTTAAATTATTATTGTATTAGAATTCTATATTAGTAGTTTAAAATATTGGAATGCAGCTGTTGAATTAACAGTGTTATAATGGTATGTTTTTATTGAAGTTATTATGGTAATTGTTATATTATTCGTAATATTGTTTAAATTCTATAGGAGATATATTGAGATTAAGTTTGTTAGTTTTATTGAGTCTAGGTAATTTCAAACGATTGTATAGTTAAGGATATTTTTATTATAACGGTAATATGTTAAATGTTATGTTGTGAATTGATTTTACGTGATTAAATTTTTTATTTTTGATGATAATTTTATTCGTTGTTTGTGGAAGTGTTTATTTGTGGGTAAAAAATGATGAAGATGTATTCTCTTTTACGAGTGATTTTGTTAGATAATCTGAATATTTTTGAAGAGTGGAGCTTAAGGTGGGTTAAAAGTGTTAAAAATTTTTATGGTTAATTATACTCAGTGTTGTTGAGGGAGTTAAAGAAAAATCGTTTTTTTGGTTTTTAAGATTATGTACGTTAGCTGTTTTTTCTATTGTTT

In [74]:
# Check to make sure file was properly written
arr = np.load('metrop_nwith_motifs_max_objc_cull.npy')
print(arr.shape)
print(convert_one_hot_seq_to_seq(arr[0][0]))
print(arr[-1][-1])

(4, 20, 1000, 4)
ATGGTATTTTAGTTGTTTTTGGGCAGTATATTAAAATTAATATTCGGGTGGTTCTTTTAGTTTTAATGAGACGATAGGAGATATTGTAATAAAATATTTTTTGAAATGATTCGTTTTAGTAGAGAATTATAGAAATGTTATAGATTAGGTGTTGTATGGATTGAGATGTATAAAGTTAGTATATATATTTATTCTATCTTTAATGTAATCAATGTTGATAGGGTGTTTGTCTCAGATATAATTTGGTTTTATAGTTATTTGTGATTTGAATGGCAAGTTAAATTATTATTGTATTAGAATTCTATATTAGTAGTTTAAAATATTGGAATGCAGCTGTTGAATTAACAGTGTTATAATGGTATGTTTTTATTGAAGTTATTATGGTAATTGTTATATTATTCGTAATATTGTTTAAATTCTATAGGAGATATATTGAGATTAAGTTTGTTAGTTTTATTGAGTCTAGGTAATTTCAAACGATTGTATAGTTAAGGATATTTTTATTATAACGGTAATATGTTAAATGTTATGTTGTGAATTGATTTTACGTGATTAAATTTTTTATTTTTGATGATAATTTTATTCGTTGTTTGTGGAAGTGTTTATTTGTGGGTAAAAAATGATGAAGATGTATTCTCTTTTACGAGTGATTTTGTTAGATAATCTGAATATTTTTGAAGAGTGGAGCTTAAGGTGGGTTAAAAGTGTTAAAAATTTTTATGGTTAATTATACTCAGTGTTGTTGAGGGAGTTAAAGAAAAATCGTTTTTTTGGTTTTTAAGATTATGTACGTTAGCTGTTTTTTCTATTGTTTTGTTTTAATTTATAGAAGTAATAGTCAAGTAGAAGTAGAGAAGATAGTGTTATTTTATGCAATGTCATGTAGGATCTTATTATGTTGAGTTATAATTTTAGGAGATTAATAGTTTTTTAGAAGGTAATGTATCCTATTAGATTGTTATATAATAGAATTAAGGACGTAA

In [0]:
from google.colab import files
# For some reason, run this twice to get it to work in Google colab. It consistently fails the first time and consistently works the second.
for a_file in files_to_use:
  files.download(a_file + ".npy")